
# SQL Exercises - DVD Rental Store

<br>

1. [Disclaimer](#disclaimer)
1. [Relevant Information](#info)
1. [Imports](#imports)
1. [Connections](#connection)
1. [Schema](#schema)
1. [Exercises](#Exercises)
    - [Median of films](#median)
    - [Filtering amount of films per price](#filter)
    - [Month over Month Customers](#mom)
    - [Monthly Active Customers, Returning Customers, Churn](#mau)
    - [Cummulative Sum](#cumsum)
    - [Moving Averages](#moving_averages)
    - [Ranking](#ranking)

<a id=disclaimer></a>

## Disclaimer
***

<div class="span5 alert alert-danger">
    <b>Note:</b> For these exercises I will be using a dataset that represents a DVD rental store. The dataset can be found in <a href=https://www.postgresqltutorial.com/postgresql-sample-database/>this page of postgresqltutorial.com</a>
</div>

[Completely Uninstall & Install PostgreSQL](https://medium.com/@bitadj/completely-uninstall-and-reinstall-psql-on-osx-551390904b86)

**About the exercises** 
- I will be using PostgreSQL, this is a different SQL flavour from the options you have in HackerRank.
- I will decide the questions I will try to answer. 
- I have used a lot of what was created by Zachary Thomas in his post [The Best Medium-Hard Data Analyst SQL Interview Questions](https://quip.com/2gwZArKuWk7W), there are great questions and great solutions to draw insights from.


<a id=info></a>

## Relevant Information
***

Here are some of the basic commands for macOS users

- `brew install postgresql` --> will install postgresql
- `brew services restart postgresql` --> will restart postgresql
- `initdb /usr/local/var/postgres` --> will point to the data directory
- `psql -U postgres` --> will ask for the password to enter your database
- `\du` --> will show the users
- `\l` --> will show the existing db
- `CREATE DATABASE hackerrank;` --> will create the database with the name leetcode (see complete syntax below)
- `\c hackerrank` --> will enter the database
- `\q` --> will close the connection to Postgres
- `CREATE TABLE tb_name;` --> Will create a table in your database
- `DROP TABLE tb_name;` --> Will delete a table from your database

**Complete syntax to create database**<br><br>
`CREATE DATABASE db_name
OWNER =  role_name
TEMPLATE = template
ENCODING = encoding
LC_COLLATE = collate
LC_CTYPE = ctype
TABLESPACE = tablespace_name
CONNECTION LIMIT = max_concurrent_connection`

<a id=imports></a>

## Imports
***

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy

In [2]:
from sqlalchemy import Table, Column, Integer, String, MetaData, VARCHAR, insert, update
from sqlalchemy.orm import sessionmaker

<a id=connection></a>

## Connection
***

In [3]:
from config import config
params = config()

In [4]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_ADDRESS = params['host']
POSTGRES_PORT = params['port']
POSTGRES_USERNAME = params['username']
POSTGRES_PASSWORD = params['password']
POSTGRES_DBNAME = params['database']

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                                                                                        password=POSTGRES_PASSWORD,
                                                                                        ipaddress=POSTGRES_ADDRESS,
                                                                                        port=POSTGRES_PORT,
                                                                                        dbname=POSTGRES_DBNAME))
# Create the connection
engine = create_engine(postgres_str) 
Session = sessionmaker(bind=engine)
session = Session()

<a id=schema></a>

## Schema

<br>
<img src="img/schema.png" style="width: 500px;"/>

**Note** I create a dictionary with the table and fields to have them readily available when needed

In [5]:
tables = ['category','inventory', 'customer', 'film_category', 'rental', 'address', 
          'film', 'payment', 'staff', 'city', 'country', 'store', 'actor', 'film_actor', 'language']

In [6]:
dic = dict()
for table in tables:
    df = pd.read_sql_query('SELECT * FROM {}'.format(table), engine)
    fields = df.columns
    dic[table] = fields.to_list()


In [7]:
for key, value in dic.items():
    print(key,value)

category ['category_id', 'name', 'last_update']
inventory ['inventory_id', 'film_id', 'store_id', 'last_update']
customer ['customer_id', 'store_id', 'first_name', 'last_name', 'email', 'address_id', 'activebool', 'create_date', 'last_update', 'active']
film_category ['film_id', 'category_id', 'last_update']
rental ['rental_id', 'rental_date', 'inventory_id', 'customer_id', 'return_date', 'staff_id', 'last_update']
address ['address_id', 'address', 'address2', 'district', 'city_id', 'postal_code', 'phone', 'last_update']
film ['film_id', 'title', 'description', 'release_year', 'language_id', 'rental_duration', 'rental_rate', 'length', 'replacement_cost', 'rating', 'last_update', 'special_features', 'fulltext']
payment ['payment_id', 'customer_id', 'staff_id', 'rental_id', 'amount', 'payment_date']
staff ['staff_id', 'first_name', 'last_name', 'address_id', 'email', 'store_id', 'active', 'username', 'password', 'last_update', 'picture']
city ['city_id', 'city', 'country_id', 'last_updat

<a id=Exercises></a>

## Exercises

<a id=median></a>

<div class="span5 alert alert-info">
    <h3> Median of inventory</h3>

**Information:** Get the median of films this dvd rental company has on stocks. Use the `inventory` table.
</div>

In [8]:
pd.read_sql_query('''
WITH cte as (SELECT film_id, COUNT(film_id) as counter 
            FROM inventory 
            GROUP BY film_id 
            ORDER BY counter )

SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY counter) as median 
FROM cte

;''', engine)


,median
0,5.0


<a id=filter></a>

<div class="span5 alert alert-info">
    <h3> Number of films per rental price</h3>

**Information:** Find the number of films in the dvd company for each rental price (use the `film` table and the `FILTER`function) <br>
</div>

In [9]:
pd.read_sql_query('''
SELECT count(*) count_all,
       count(*) FILTER(WHERE rental_rate=0.99) price_099,
       count(*) FILTER(WHERE rental_rate=2.99) price_299,
       count(*) FILTER(WHERE rental_rate=4.99) price_499       
  FROM film;

;''', engine)

,count_all,price_099,price_299,price_499
0,1000,341,323,336


<a id=mom></a>

<div class="span5 alert alert-info">
    <h3> Month over Month number of new customers</h3>

**Information:** Check how many new customers and the percent increase month to month<br>
</div>

**Note:** unfortunately there are no different creation dates, only one month

In [10]:
pd.read_sql_query('''
SELECT DATE_TRUNC('month', t1.create_date), DATE_TRUNC('month', t2.create_date)
FROM customer t1
JOIN customer t2 ON t1.customer_id = t2.customer_id AND DATE_TRUNC('month', t1.create_date) = DATE_TRUNC('month', t2.create_date) + interval '1 month'  
;''', engine)

,date_trunc,date_trunc


<a id=mau></a>

<div class="span5 alert alert-info">
    <h3> Monthly Active Users (MAU), Retained Users per Month, Churned Users, </h3>

**Information:** Questions from [Zachary Thomas](https://quip.com/2gwZArKuWk7W) <br>
    - Task1: Get the total number of users (people who rented a movie each month)<br>
    - Task2: Write a query that gets the number of retained users per month. In this case, retention for a given month is defined as the number of users who rented in that month who also rented in the immediately previous month. <br>
    - Task3: Now we’ll take retention and turn it on its head: Write a query to find many users last month did not come back this month. i.e. the number of churned users.  <br>
</div>

In [11]:
dic['rental']

['rental_id',
 'rental_date',
 'inventory_id',
 'customer_id',
 'return_date',
 'staff_id',
 'last_update']

In [12]:
pd.read_sql_query('''
SELECT DATE_TRUNC('month', rental_date) as trunc, COUNT(DISTINCT(customer_id))
FROM rental
GROUP BY trunc
ORDER BY trunc
;''', engine)

,trunc,count
0,2005-05-01,520
1,2005-06-01,590
2,2005-07-01,599
3,2005-08-01,599
4,2006-02-01,158


In [13]:
pd.read_sql_query('''
SELECT DATE_PART('month', DATE_TRUNC('month', t1.rental_date)) as Month, COUNT(DISTINCT(t1.customer_id))
FROM rental as t1
JOIN rental as t2 ON t1.customer_id = t2.customer_id AND DATE_TRUNC('month', t1.rental_date) = DATE_TRUNC('month', t2.rental_date) + interval '1 month'
GROUP BY DATE_TRUNC('month', t1.rental_date)
;''', engine)

,month,count
0,6.0,512
1,7.0,590
2,8.0,599


**Note:** February 2006 does not have a row because we are comparing with the previous "real" month, not the last recorded month

In [14]:
pd.read_sql_query('''
SELECT DATE_TRUNC('month', t1.rental_date), COUNT(DISTINCT(t1.customer_id))
FROM rental as t1
LEFT JOIN rental as t2 ON t1.customer_id = t2.customer_id AND DATE_TRUNC('month', t1.rental_date) = DATE_TRUNC('month', t2.rental_date) + interval '1 month'
WHERE t2.customer_id IS NULL
GROUP BY DATE_TRUNC('month', t1.rental_date)
;''', engine)

,date_trunc,count
0,2005-05-01,520
1,2005-06-01,78
2,2005-07-01,9
3,2006-02-01,158


<a id=cumsum></a>

<div class="span5 alert alert-info">
    <h3> Cummulative Sum</h3>

**Information:** Find the total revenue generated each day and provide the cummulative sum of the previous days<br>
</div>

Good resource: [link](http://www.dba-oracle.com/t_advanced_sql_windowing_clause.htm)

In [15]:
pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day',rental_date) as date, SUM(amount) as daily_total
FROM rental t1
JOIN payment t2 ON t1.rental_id = t2.rental_id
GROUP BY DATE_TRUNC('day',rental_date)
ORDER BY date)

SELECT date, daily_total, SUM(daily_total) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as total
FROM cte
;''', engine).head()

,date,daily_total,total
0,2005-06-14,41.89,41.89
1,2005-06-15,1179.97,1221.86
2,2005-06-16,1191.11,2412.97
3,2005-06-17,1158.19,3571.16
4,2005-06-18,1284.99,4856.15


**The solution bellow is provided by Thomas on his website**

In [16]:
pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day',rental_date) as date, SUM(amount) as daily_total
FROM rental t1
JOIN payment t2 ON t1.rental_id = t2.rental_id
GROUP BY DATE_TRUNC('day',rental_date)
ORDER BY date)

SELECT date, SUM(daily_total) OVER (ORDER BY date ASC) as cumulative_cf 
FROM cte 
ORDER BY date ASC

;''', engine).head()

,date,cumulative_cf
0,2005-06-14,41.89
1,2005-06-15,1221.86
2,2005-06-16,2412.97
3,2005-06-17,3571.16
4,2005-06-18,4856.15


<a id=moving_averages></a>

<div class="span5 alert alert-info">
    <h3> Moving Averages</h3>

**Information:** <br>
    - Exercise 1: Find the 7 day moving average of the number of daily rentals<br>
    - Exercise 2: Find the cummulative average of daily rentals<br>
</div>

In [17]:
dic['rental']

['rental_id',
 'rental_date',
 'inventory_id',
 'customer_id',
 'return_date',
 'staff_id',
 'last_update']

### Part 1: 7 day Moving Average

In [18]:
pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day', rental_date) as day, rental_id
FROM rental),

cte2 as (SELECT day, COUNT(rental_id)
FROM cte
GROUP BY day
ORDER BY day)

SELECT day, count, AVG(count) OVER (ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as rolling_avg
FROM cte2
;''', engine).head(10)

,day,count,rolling_avg
0,2005-05-24,8,8.000000
1,2005-05-25,137,72.500000
2,2005-05-26,174,106.333333
3,2005-05-27,166,121.250000
4,2005-05-28,196,136.200000
5,2005-05-29,154,139.166667
6,2005-05-30,158,141.857143
7,2005-05-31,163,164.000000
8,2005-06-14,16,146.714286
9,2005-06-15,348,171.571429


**Check:** let's make sure this has been done correctly by doing the rolling average with pandas' `window` function

In [19]:
df = pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day', rental_date) as day, rental_id
FROM rental),

cte2 as (SELECT day, COUNT(rental_id)
FROM cte
GROUP BY day
ORDER BY day)

SELECT day, count, AVG(count) OVER (ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as SQL_MA_7
FROM cte2
;''', engine).head(10)

df['pandas_MA_7'] = df.iloc[:,1].rolling(window=7).mean()
df['difference']= df['sql_ma_7'] - df['pandas_MA_7']
df.head(10)

,day,count,sql_ma_7,pandas_MA_7,difference
0,2005-05-24,8,8.000000,NaN,NaN
1,2005-05-25,137,72.500000,NaN,NaN
2,2005-05-26,174,106.333333,NaN,NaN
3,2005-05-27,166,121.250000,NaN,NaN
4,2005-05-28,196,136.200000,NaN,NaN
5,2005-05-29,154,139.166667,NaN,NaN
6,2005-05-30,158,141.857143,141.857143,0.0
7,2005-05-31,163,164.000000,164.000000,0.0
8,2005-06-14,16,146.714286,146.714286,0.0
9,2005-06-15,348,171.571429,171.571429,0.0


### Part 2: Cummulative AVG

In [20]:
pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day', rental_date) as day, rental_id
FROM rental),

cte2 as (SELECT day, COUNT(rental_id)
FROM cte
GROUP BY day
ORDER BY day)

SELECT day, count, AVG(count) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cummulative_avg 
FROM cte2
;''', engine).head()

,day,count,cummulative_avg
0,2005-05-24,8,8.000000
1,2005-05-25,137,72.500000
2,2005-05-26,174,106.333333
3,2005-05-27,166,121.250000
4,2005-05-28,196,136.200000


**Check:** let's make sure this has been done correctly by doing the cummulative average with pandas' `expanding` function

In [21]:
df = pd.read_sql_query('''
WITH cte as (SELECT DATE_TRUNC('day', rental_date) as day, rental_id
FROM rental),

cte2 as (SELECT day, COUNT(rental_id)
FROM cte
GROUP BY day
ORDER BY day)

SELECT day, count, AVG(count) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cummulative_avg 
FROM cte2
;''', engine)

df['pandas_cum_avg'] = df['count'].expanding(min_periods=1).mean()
df['difference'] = df['cummulative_avg'] - df['pandas_cum_avg']
df.head()

,day,count,cummulative_avg,pandas_cum_avg,difference
0,2005-05-24,8,8.000000,8.000000,0.0
1,2005-05-25,137,72.500000,72.500000,0.0
2,2005-05-26,174,106.333333,106.333333,0.0
3,2005-05-27,166,121.250000,121.250000,0.0
4,2005-05-28,196,136.200000,136.200000,0.0


<a id=ranking></a>

<div class="span5 alert alert-info">
    <h3> Ranking</h3>

**Information:**<br>
    - Exercise 1: Rank the PG-13 movies from longest to shortest and select the rows with the titles that contain the longest and shortest movies. Note: if there are two movies that have the same lenght, these should be given the rank 1 and the next longest films show receive rank 2 not 3<br>
    - Exercise 2: Find the monthly sales by employee, rank them in order from largest total sales to lowest total sales. Return: the rank, the total_sales, the month, the staff_id, the name, the last name.
</div>

### Part 1: Ranking of Longest and shortes PG-13 Movies

In [22]:
dic['film']

['film_id',
 'title',
 'description',
 'release_year',
 'language_id',
 'rental_duration',
 'rental_rate',
 'length',
 'replacement_cost',
 'rating',
 'last_update',
 'special_features',
 'fulltext']

In [23]:
pd.read_sql_query('''
WITH cte AS (SELECT title, rating, length, DENSE_RANK() OVER (ORDER BY length DESC) as dense_rank 
FROM film
WHERE rating ='PG-13' )

SELECT *
FROM cte
WHERE dense_rank = 1 OR dense_rank = (SELECT dense_rank FROM cte ORDER by dense_rank DESC LIMIT 1)
;''', engine)

,title,rating,length,dense_rank
0,Chicago North,PG-13,185,1
1,Gangs Pride,PG-13,185,1
2,Pond Seattle,PG-13,185,1
3,Labyrinth League,PG-13,46,116
4,Ridgemont Submarine,PG-13,46,116
5,Kwai Homeward,PG-13,46,116


### Part 2: Ranking of Employees salaries

In [24]:
print(dic['payment'])
print(dic['staff'])


['payment_id', 'customer_id', 'staff_id', 'rental_id', 'amount', 'payment_date']
['staff_id', 'first_name', 'last_name', 'address_id', 'email', 'store_id', 'active', 'username', 'password', 'last_update', 'picture']


In [25]:
pd.read_sql_query('''
SELECT RANK() OVER(ORDER BY total_sales DESC), total_sales, month, t.staff_id, first_name, last_name
FROM (SELECT staff_id, SUM(amount) as total_sales, DATE_TRUNC('month',payment_date) as month
    FROM payment
    GROUP BY staff_id, month) as t
JOIN staff ON t.staff_id = staff.staff_id
;''', engine)

,rank,total_sales,month,staff_id,first_name,last_name
0,1,14479.10,2007-04-01,2,Jon,Stephens
1,2,14080.36,2007-04-01,1,Mike,Hillyer
2,3,12109.73,2007-03-01,2,Jon,Stephens
3,4,11776.83,2007-03-01,1,Mike,Hillyer
4,5,4191.00,2007-02-01,2,Jon,Stephens
5,6,4160.84,2007-02-01,1,Mike,Hillyer
6,7,280.09,2007-05-01,2,Jon,Stephens
7,8,234.09,2007-05-01,1,Mike,Hillyer


<div class="span5 alert alert-info">
    <h3> Complex Query</h3>

**Information:** Find how many movies a each client has rented before returning a previous movie. Return their customer_id, their full name as one column, and the number of movies they have not returned before renting a new one. Order the results by this final count.<br>
</div>

In [26]:
pd.read_sql_query('''
WITH cte AS (SELECT  t1.customer_id, t1.rental_id as rental_id, t1.return_date,t2.rental_id as rental_id2, t2.rental_date
FROM rental t1
JOIN rental t2 ON t1.customer_id = t2.customer_id AND t1.rental_id != t2.rental_id
WHERE t1.return_date > t2.rental_date),

cte2 as (SELECT DISTINCT(rental_id), customer_id
FROM cte)

SELECT cte2.customer_id, CONCAT(first_name,' ', last_name) as full_name, COUNT(*)
FROM cte2
LEFT JOIN customer ON cte2.customer_id = customer.customer_id
GROUP BY cte2.customer_id, full_name
ORDER BY count DESC

;''', engine)

,customer_id,full_name,count
0,148,Eleanor Hunt,45
1,526,Karl Seal,45
2,144,Clara Shaw,42
3,236,Marcia Dean,41
4,197,Sue Peters,40
...,...,...,...
594,110,Tiffany Jordan,14
595,281,Leona Obrien,14
596,136,Anita Morales,14
597,61,Katherine Rivera,13
